# Ollama embedding similarity

## Imports

In [ ]:
import requests
import ollama

## Models 

- Phi3: context window 8k
- nomic: 8k context 768 dimensions

In [ ]:
def embed(prompt:str, model='nomic-embed-text'):
    vector = ollama.embeddings(model=model, prompt=prompt)['embedding']
    #print(len(vector))
    return vector

def completion(prompt:str, model='phi3'):
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': prompt,
    },
    ])
    print(response['message']['content'])

def rest_embed(prompt:str, model='nomic-embed-text',endpoint='http://localhost:11434/api/embeddings'):
    payload = {
        "model": model,
        "prompt": prompt
    }
    req = requests.post(endpoint, json=payload)
    req.raise_for_status()
    vector = req.json()['embedding']
    #print(len(vector))
    return vector

def rest_completion(prompt:str, model='phi3',endpoint='http://localhost:11434/api/chat'):
    payload = {
        "model": model,
        "messages": [
            {
                'role': 'user',
                'content': prompt,
            }
        ],
        "stream": False
    }
    req = requests.post(endpoint, json=payload)
    req.raise_for_status()
    return req.json()['message']['content']

In [ ]:
food = [
    "Tacos al pastor",
    "Pizza Margherita",
    "Picanha na brasa",
    "Fettuccine alla carbonara",
    "Mole de pollo",
    "Asado de tira",
    "Tiramisu",
    "Brigadeiro",
    "Arroz con leche"
]

characters = [
    "Harry Potter",
    "Ron Weasley",
    "Hermione Granger",
    "Albus Dumbledore",
    "Frodo Baggins",
    "Gandalf",
    "Aragorn",
    "Legolas",
]

history = [
    """Guatemala's history is rich and complex, marked by the remarkable achievements of the Maya civilization and the dramatic changes following Spanish conquest. The Maya, known for their monumental architecture, advanced mathematics, and astronomical systems, thrived in the region for thousands of years before the arrival of the Spanish in the 16th century. After nearly three centuries under Spanish rule, Guatemala gained independence in 1821, followed by periods of political turmoil, including a 36-year civil war. Today, Guatemala is a republic that continues to navigate the challenges of its historical legacy.""",    
    """The history of the United States is a rich tapestry that spans thousands of years, beginning with the arrival of the first people in the Americas around 15,000 BC. These indigenous cultures underwent significant transformations, particularly with the arrival of Europeans in the 16th century. The subsequent period saw the struggle for independence, the formation of a new nation, and the expansion across the continent. Key events such as the Civil War, the Industrial Revolution, and participation in global conflicts have shaped the modern era of the United States, reflecting a history of resilience and change."""
]

vectordb = []
for f in history:
    v = {"chunk":f, "vector":embed(f)}
    print(v)
    vectordb.append(v)

In [ ]:
def dot_product(v1, v2):
    return sum(map(lambda x: x[0] * x[1], zip(v1, v2)))

def magnitude(vector):
    return sum(map(lambda x: x**2, vector)) ** 0.5

def cosine_similarity(v1, v2):
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2))

In [ ]:
v1 = embed("Was there slavery in the united states?")
for v in vectordb:
    v2 = v['vector']
    print(v['chunk'][:10], cosine_similarity(v1, v2))
